# Assistant-06<br/>Code Interpreter & Function calling

## Trading Assistant that can calculate portfolio values and email a report

## Credits

- https://mer.vin/2023/11/openai-assistants-api-function-calling/

## Email functionality

- Implemented using Logic Apps with an Http POST trigger

### Get an OpenAI client and an Assistant

**Note:** When configuring the OpenAI client, the API version, the model version, and endpoint are "new".

In [1]:
import time
import oaihelper as helper
import yfinance as yf


## Get an OpenAI client
client = helper.get_openai_client(api_key=helper.api_KEY,
        api_version=helper.api_version,
        azure_endpoint=helper.api_URI,)

### Get the latest stock price by ticker symbol using Yahoo Finance

In [2]:
def get_stock_price(symbol: str) -> float:
    try:
        stock = yf.Ticker(symbol)
        price = stock.history(period="1d")['Close'].iloc[-1]    
        return price
    except:
        return 0

### Prepare the tools for function calling

In [3]:
tools_list = [
{"type": "code_interpreter"},
{"type": "function",
    "function": {

        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
},
{
    "type": "function",
    "function": {
        "name": "send_email",
        "description": "Sends an email to a recipient(s).",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {
                    "type": "string",
                    "description": "The email(s) the email should be sent to."
                },
                "content": {
                    "type": "string",
                    "description": "The content of the email."
                }
            },
            "required": ["to", "content"]
        }
    }
}]

In [4]:
csv_file = helper.upload_file(client,"../data/finance/fta-portfolio.csv")

Added file:  assistant-PG6QBBpXAVnE2wTsaiycOnvd 1


### Create the assistant

In [5]:
helper.clear_shelves()

## Create an assistant
stock_assistant = helper.create_assistant(client,
                                                  name="fta-Securities Trading Assistant",
                                                  instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly.", 
                                                  tools=tools_list,                                                  
                                                  model=helper.gpt_deployment_name,
                                                  file_ids=[csv_file.id]
                                                  )

Added assistant:  asst_9XFFufWoLki4SooA1OLVU9GD 1


### Define the function calling delegate

In [6]:
def function_calling_delegate(client, thread, run):
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json
    for action in required_actions["tool_calls"]:
        func_name = action['function']['name']
        arguments = json.loads(action['function']['arguments'])
        
        if func_name == "get_stock_price":
            output = get_stock_price(symbol=arguments['symbol'])
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": output
            })
        elif func_name == "send_email":
            print("Sending email...")
            email_to = arguments['to']
            email_content = arguments['content']
            helper.send_email(email_to,email_content)
            
            tool_outputs.append({
                "tool_call_id": action['id'],
                "output": "Email sent"
            })
        else:
            raise ValueError(f"Unknown function: {func_name}")
        
    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )

### Process the message, function calling leveraging the latest stock prices, print the results and dispose of the objects

In [7]:
helper.generate_response(client,stock_assistant,"Based on the provided portfolio, please email me an HTML formatted report to alemor@microsoft.com with the details for each stock based on the latest stock prices, and list the best and worst performing stocks in my portfolio.", "123", "Alex", function_calling_delegate)

Creating new thread for Alex with user_id 123
Added thread:  thread_TypTapr7ZuM5RzRrsnmvPhD2 1
Function Calling
{'tool_calls': [{'id': 'call_bYSlpRF0Zo9JbXWzMnlGtPfm', 'function': {'arguments': '{"symbol": "AAPL"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_KXPtUHk3J6OsIRI1OQ9baxPx', 'function': {'arguments': '{"symbol": "MSFT"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_uUHEBA1QO5MgaYMvBD8VF8UD', 'function': {'arguments': '{"symbol": "GOOGL"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_rbQtnCobifkEMOiZl1HHFlR2', 'function': {'arguments': '{"symbol": "AMZN"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_eGPFemW2ryEZvP2LMCR5YFHT', 'function': {'arguments': '{"symbol": "TSLA"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_0T0Ucotwew9JhUDL58VTc4C5', 'function': {'arguments': '{"symbol": "NFLX"}', 'name': 'get_stock_price'}, 'type': 'function'}, {'id': 'call_vvcJslXqaUJzwjYkXtPIJDm2', 'function

"The HTML formatted report has been successfully emailed to alemor@microsoft.com. If there's anything more I can assist you with, please let me know!"

### Delete all demo objects

In [8]:
helper.cleanup(client)

Deleting:  1  assistants.
AssistantDeleted(id='asst_9XFFufWoLki4SooA1OLVU9GD', deleted=True, object='assistant.deleted')
Deleting:  1  threads.
ThreadDeleted(id='thread_TypTapr7ZuM5RzRrsnmvPhD2', deleted=True, object='thread.deleted')
Deleting:  1  files.
FileDeleted(id='assistant-PG6QBBpXAVnE2wTsaiycOnvd', deleted=True, object='file')
